remova http, emoji, url fonksiyonları çalıştırıldı.
sonuç bir tık yükseldi ama yeterli değil!
spellchekr (yazım yanlışı kontolü çalıştırıldı ve sonucu beklenıyor yavaş çalıştığı için.

In [27]:
import numpy as np
import pandas as pd
import re

# collection of machine learning algorithms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
# Common Model Helpers
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn import model_selection
import pylab as pl
from sklearn.metrics import roc_curve
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.stats import norm

pd.set_option("display.max_rows",10000)  # KISALTMA ENGELLEME
pd.set_option("display.max_columns",10000)  # KISALTMA ENGELLEME

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv') 

In [28]:
from nltk.corpus import wordnet
from collections import Counter
def get_part_of_speech(word):
  probable_part_of_speech = wordnet.synsets(word)
  pos_counts = Counter()
  pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
  pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
  pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
  pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )
  
  most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
  return most_likely_part_of_speech

In [29]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)



In [30]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)


In [31]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)



In [32]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [33]:
submission = pd.read_csv('sample_submission.csv') 

In [34]:
train.info()

train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [35]:
train['text'] = train['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
test['text'] = test['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [36]:
train['text']=train['text'].apply(lambda x : remove_URL(x))
test['text']=test['text'].apply(lambda x : remove_URL(x))

In [37]:
train['text']=train['text'].apply(lambda x : remove_html(x))
test['text']=test['text'].apply(lambda x : remove_html(x))

In [38]:
train['text']=train['text'].apply(lambda x: remove_emoji(x))
test['text']=test['text'].apply(lambda x: remove_emoji(x))

In [39]:
train['text']=train['text'].apply(lambda x : correct_spellings(x))
test['text']=test['text'].apply(lambda x : correct_spellings(x))

KeyboardInterrupt: 

In [ ]:
train.head()

In [ ]:
train['text'] = train['text'].str.replace('[^\w\s]','')
test['text'] = test['text'].str.replace('[^\w\s]','')
train['text'] = train['text'].str.replace('[\d\_]','')
test['text'] = test['text'].str.replace('[\d\_]','')

In [ ]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')

In [208]:
train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
test['text'] = test['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

In [209]:
sil = pd.Series(' '.join(train['text']).split()).value_counts()[-50:]
sil = pd.Series(' '.join(test['text']).split()).value_counts()[-50:]

train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))
test['text'] = test['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))

In [210]:
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer ()

train['text'] = train['text'].apply(lambda x: " ".join(lemmatizer.lemmatize(i,get_part_of_speech(i)) for i in x.split()))
test['text'] = test['text'].apply(lambda x: " ".join(lemmatizer.lemmatize(i,get_part_of_speech(i)) for i in x.split()))

In [211]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deed reason earthquake may allah forgive u,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,resident ask shelter place notify officer evac...,1
3,6,NaN,NaN,people receive wildfire evacuation order calif...,1
4,7,NaN,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [212]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(train["text"],
                                                                   train["target"], 
                                                                    random_state = 1)

In [213]:
train_x.head()


5242    sb new deepwater horizon oil spill distributio...
4537                 rvacchianonydn surprise arent injure
6267    rt tonyhsieh person dance rain likely walk sto...
5486    yet another company try censor internet reddit...
985     mattbez oh im bag body bangin im say shes go g...
Name: text, dtype: object

In [214]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF Idf WordLeVel

In [215]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit_transform(train_x)

<5709x12088 sparse matrix of type '<class 'numpy.float64'>'
	with 50329 stored elements in Compressed Sparse Row format>

In [216]:
test1_tf_idf_word = tf_idf_word_vectorizer.transform(test1)

In [217]:
test1_tf_idf_word.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [218]:
test1.head()

0                            happen terrible car crash
1    hear earthquake different city stay safe everyone
2    forest fire spot pond goose flee across street...
3                    apocalypse light spokane wildfire
4                   typhoon soudelor kill china taiwan
Name: text, dtype: object

In [219]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

 # TF IDF N-gram

In [183]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range = (2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [184]:
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

# TF ıdf Chars

In [135]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [136]:
x_train_tf_idf_chars = tf_idf_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

# LogisticRegression

In [92]:
from sklearn.linear_model import LogisticRegression

In [190]:
loj = LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

TypeError: fit() missing 1 required positional argument: 'y'

In [139]:
loj = LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7600055111600992


In [220]:
loj = LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word,train_y)

y_pred_word_loj = loj_model.predict(x_test_tf_idf_word)

accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.7710140534582529


# MultinomialNB

In [94]:
from sklearn.naive_bayes import MultinomialNB

In [182]:
nb = MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.7657288509231194


In [186]:
nb = MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.6906668503720035


In [144]:
nb = MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.6727996693303939


# RF

In [147]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.7689335905208046


In [150]:
rf = RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.6786001653348028


In [151]:
rf = RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7279636263433453


In [106]:
y_pred = nb_model.predict(test1_tf_idf_word)

In [101]:
#print(y_pred)

[1 0 1 ... 1 1 1]


In [107]:
submission= submission[["id"]]

In [108]:
y_pred1 = pd.DataFrame(data = y_pred, index = range(3263), columns=['target'])

In [110]:
final_submission=pd.concat([submission,y_pred1],axis=1)

In [112]:
#final_submission.to_csv('06062020_3.csv',index=False)

In [104]:
#final_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [193]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, y_pred_word_loj)

array([[984, 107],
       [264, 549]], dtype=int64)